In [33]:
import pandas as pd

In [34]:
df = pd.read_csv("ventas_por_producto.csv")

In [35]:
df.columns

Index(['id_venta', 'id_producto', 'nombre_producto_venta', 'cantidad',
       'precio_unitario_venta', 'importe', 'nombre_producto_producto',
       'precio_unitario_producto', 'fecha', 'id_cliente', 'ciudad_Alta Gracia',
       'ciudad_Carlos Paz', 'ciudad_Cordoba', 'ciudad_Mendiolaza',
       'ciudad_Rio Cuarto', 'ciudad_Villa Maria', 'cat_Alimentos',
       'cat_Limpieza', 'medio_pago_efectivo', 'medio_pago_qr',
       'medio_pago_tarjeta', 'medio_pago_transferencia', 'outlier',
       'rotacion_dias', 'mes', 'total_cantidad', 'total_importe',
       'total_transacciones', 'rotacion_promedio'],
      dtype='object')

In [36]:
# Cantidad promedio de compra
df['cantidad_promedio_compra'] = df['total_cantidad'] / df['total_transacciones']

# Frecuencia de compra (transacciones totales por producto)
df['frecuencia_compra'] = df['total_transacciones']

# O frecuencia mensual (si quieres normalizarla)
df['frecuencia_compra_mensual'] = df['total_transacciones'] / df['mes'].nunique()


In [37]:
df

,id_venta,id_producto,nombre_producto_venta,cantidad,precio_unitario_venta,importe,nombre_producto_producto,precio_unitario_producto,fecha,id_cliente,...,outlier,rotacion_dias,mes,total_cantidad,total_importe,total_transacciones,rotacion_promedio,cantidad_promedio_compra,frecuencia_compra,frecuencia_compra_mensual
0,57,41,Aceite de Girasol 1L,4,860,3440,Aceite de Girasol 1L,860,2024-01-10,34,...,False,0.0,2024-01,19,16340,6,22.833333,3.166667,6,1.000000
1,86,41,Aceite de Girasol 1L,3,860,2580,Aceite de Girasol 1L,860,2024-01-10,40,...,False,0.0,2024-01,19,16340,6,22.833333,3.166667,6,1.000000
2,72,41,Aceite de Girasol 1L,4,860,3440,Aceite de Girasol 1L,860,2024-02-17,26,...,False,38.0,2024-02,19,16340,6,22.833333,3.166667,6,1.000000
3,64,41,Aceite de Girasol 1L,2,860,1720,Aceite de Girasol 1L,860,2024-03-07,58,...,False,19.0,2024-03,19,16340,6,22.833333,3.166667,6,1.000000
4,80,41,Aceite de Girasol 1L,5,860,4300,Aceite de Girasol 1L,860,2024-03-25,54,...,False,18.0,2024-03,19,16340,6,22.833333,3.166667,6,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,22,9,Yerba Mate Suave 1kg,2,3878,7756,Yerba Mate Suave 1kg,3878,2024-05-08,64,...,False,116.0,2024-05,20,77560,6,25.000000,3.333333,6,1.000000
339,52,9,Yerba Mate Suave 1kg,4,3878,15512,Yerba Mate Suave 1kg,3878,2024-05-10,5,...,False,2.0,2024-05,20,77560,6,25.000000,3.333333,6,1.000000
340,61,9,Yerba Mate Suave 1kg,3,3878,11634,Yerba Mate Suave 1kg,3878,2024-06-01,27,...,False,22.0,2024-06,20,77560,6,25.000000,3.333333,6,1.000000
341,100,9,Yerba Mate Suave 1kg,4,3878,15512,Yerba Mate Suave 1kg,3878,2024-06-08,69,...,False,7.0,2024-06,20,77560,6,25.000000,3.333333,6,1.000000


In [38]:
cols_drop = [
    'id_venta',
    'id_cliente',
    'fecha',
    'mes',
    'nombre_producto_venta',
    'nombre_producto_producto',
    'precio_unitario_venta',
    'rotacion_dias',
    'rotacion_promedio',
]
df_clean = df.drop(columns=cols_drop)

In [39]:
dummy_cols = [
    'ciudad_Alta Gracia', 'ciudad_Carlos Paz', 'ciudad_Cordoba',
    'ciudad_Mendiolaza', 'ciudad_Rio Cuarto', 'ciudad_Villa Maria',
    'cat_Alimentos', 'cat_Limpieza',
    'medio_pago_efectivo', 'medio_pago_qr',
    'medio_pago_tarjeta', 'medio_pago_transferencia'
]

In [40]:
df_prod = df_clean.groupby('id_producto').agg({
    'cantidad': 'sum',
    'importe': 'sum',
    'precio_unitario_producto': 'mean',
    'total_cantidad': 'max',
    'total_importe': 'max',
    'total_transacciones': 'max',
    **{col: 'sum' for col in dummy_cols}
}).reset_index()

In [41]:
umbral = df_prod['total_transacciones'].median()

df_prod['baja_rotacion'] = (df_prod['total_transacciones'] < umbral).astype(int)

In [42]:
df_prod = df_prod.drop(columns=['total_transacciones'])


In [43]:
df_prod

,id_producto,cantidad,importe,precio_unitario_producto,total_cantidad,total_importe,ciudad_Alta Gracia,ciudad_Carlos Paz,ciudad_Cordoba,ciudad_Mendiolaza,ciudad_Rio Cuarto,ciudad_Villa Maria,cat_Alimentos,cat_Limpieza,medio_pago_efectivo,medio_pago_qr,medio_pago_tarjeta,medio_pago_transferencia,baja_rotacion
0,1,9,21123,2347.0,9,21123,0,0,0,0,2,0,2,0,2,0,0,0,1
1,2,9,44757,4973.0,9,44757,0,0,0,0,3,0,3,0,0,1,2,0,0
2,3,4,19856,4964.0,4,19856,0,0,0,0,1,0,1,0,0,1,0,0,1
3,4,16,32528,2033.0,16,32528,1,2,0,1,1,0,5,0,1,2,0,2,0
4,5,9,42993,4777.0,9,42993,0,1,0,1,1,0,3,0,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,94,7,9926,1418.0,7,9926,1,0,0,0,1,0,0,2,0,2,0,0,1
91,95,14,22134,1581.0,14,22134,0,1,2,0,1,0,0,4,0,1,0,3,0
92,97,15,13080,872.0,15,13080,1,1,1,0,1,0,0,4,1,1,1,1,0
93,98,15,42645,2843.0,15,42645,1,1,1,2,0,1,0,6,3,1,1,1,0


In [44]:
X = df_prod.drop(columns=['baja_rotacion'])
y = df_prod['baja_rotacion']

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y   # MUY importante si hay desbalance
)

In [46]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [47]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(
    max_iter=1000,       # evitar errores de convergencia
    solver='lbfgs'       # solver estándar
)

logreg.fit(X_train_scaled, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [48]:
y_prob = logreg.predict_proba(X_test_scaled)[:, 1]
y_prob[:10]

array([9.87732761e-01, 1.38373825e-03, 2.93734936e-01, 6.75758045e-05,
       3.60760766e-01, 9.76078805e-01, 6.55327831e-03, 8.11158831e-01,
       3.40742295e-03, 2.73365044e-03])

In [49]:
resultados = X_test.copy()
resultados['prob_baja_rotacion'] = y_prob
resultados['baja_rotacion_real'] = y_test.values

resultados.head(20)

,id_producto,cantidad,importe,precio_unitario_producto,total_cantidad,total_importe,ciudad_Alta Gracia,ciudad_Carlos Paz,ciudad_Cordoba,ciudad_Mendiolaza,ciudad_Rio Cuarto,ciudad_Villa Maria,cat_Alimentos,cat_Limpieza,medio_pago_efectivo,medio_pago_qr,medio_pago_tarjeta,medio_pago_transferencia,prob_baja_rotacion,baja_rotacion_real
51,52,2,5164,2582.0,2,5164,0,0,1,0,0,0,0,1,1,0,0,0,0.987733,1
23,24,11,14355,1305.0,11,14355,2,1,0,0,1,1,5,0,0,2,1,2,0.001384,0
81,85,5,9280,1856.0,5,9280,1,0,1,0,1,0,3,0,1,1,1,0,0.293735,0
12,13,14,33362,2383.0,14,33362,0,1,1,0,3,1,6,0,1,2,2,1,0.000068,0
28,29,7,13076,1868.0,7,13076,1,0,0,2,0,0,3,0,0,0,2,1,0.360761,0
45,46,5,15180,3036.0,5,15180,0,0,0,0,0,1,1,0,1,0,0,0,0.976079,1
33,34,15,7545,503.0,15,7545,1,1,1,1,0,0,4,0,2,0,1,1,0.006553,0
47,48,3,13386,4462.0,3,13386,1,0,1,0,0,0,2,0,1,0,1,0,0.811159,1
78,82,15,35910,2394.0,15,35910,1,1,0,1,1,0,4,0,1,1,1,1,0.003407,0
9,10,8,39064,4883.0,8,39064,1,1,0,0,3,0,5,0,0,1,0,4,0.002734,0


In [50]:
y_pred = logreg.predict(X_test_scaled)

#Metricas
- Accuracy
- Precision
- Recall
- F1-Score
- Matriz de confusión


In [54]:
y_pred

array([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0])

In [ ]:
#Metrica Precision 
#REVISAR
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred)
precision


1.0

In [ ]:
#RECALL
#REVISAR 

from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred)
print("Recall:", recall)

Recall: 1.0
